# Running Report Code
This notebook walks through the analyses done for the NCST Report #: Simulating Bike-Transit Trips through BikewaySim.

Before proceeding make sure to install packages listed in the modules.txt file.

And clone this forked reporsitory with GTFS data and RAPTOR algorithm: [reidx19/transit-routing](https://github.com/reidx19/transit-routing). This forked repo already has the pre-processed MARTA GTFS data and contains a small edit to the transit-routing code used for post-processing the RAPTOR results.

This notebook has the code to generate a study area from transit stops and perform shortest path routing for using bike or walk as a first-last mile mode. There are three scripts for running these analyses:

1. find candidate stops
1. raptor pre-processing
1. raptor routing
1. raptor mapping
1. raptor stats and viz

In [ ]:
#need to add transit-routing repo to path so its modules can be imported
import sys
from pathlib import Path
sys.path.insert(0,str(Path.home() / 'Documents/GitHub/transit-routing'))

#custom modules
from find_candidate_stops import *
from create_transfers import *
from raptor_routing import *
from raptor_mapping import *
from viz_and_metrics import *

settings = {
    #changed network from marta to martalatest
    
    #these are for the pre-processing steps
    'gtfs_fp': Path.home() / 'Documents/GitHub/transit-routing/GTFS/martalatest', #filepath for processed GTFS files
    'gtfs_zip': Path.home() / 'Documents/GitHub/transit-routing/martalatest_gtfs.zip', #filepath for original zipped GTFS files
    'network_fp': Path.home() / 'Documents/TransitSimData/networks/final_network.gpkg', #fp to geopackage with a links and nodes layer
    'links_layer': 'links', # name of links layer
    'nodes_layer': 'nodes', # name of nodes layer
    'impedance': 'dist', # specify which column of the links data should be used for shortest path routing
    'service_date': date(2023,3,1),#date(2022, 11, 24), #select day to get service, a weekday was used for this study
    'crs': 'epsg:2240', # the desired projected CRS to use
    'thresh': 2.5 * 5280, #set candidate stop threshold distance in units of CRS (using bike service area from tcqsm page 5-20)
    'tazs_fp': Path.home() / 'Documents/NewBikewaySimData/Data/ARC/Model_Traffic_Analysis_Zones_2020.geojson', # filepath of TAZs or origins/POIs
    'keyid': 'OBJECTID', #column with the unique taz/origin id in it
    
    #this for making the transfers.txt file need for raptor
    'transfer_time': 2, # allowed transfer time in minutes (DOES NOT INLCUDE WAIT TIME)
    'walk_spd': 2.5, # assumed walking speed for transfers in miles per hour
    
    #these are specific to RAPTOR
    'NETWORK_NAME': 'martalatest',#'marta',
    'MAX_TRANSFER': 2, # no more than 1 transfer
    'WALKING_FROM_SOURCE': 0, # if true (1), person can walk to a different transit station from start station
    'CHANGE_TIME_SEC': 30, # time to enter/exit vehicle (NOT WAIT TIME)
    'PRINT_ITINERARY': 0, # if running individual raptor trips, this prints the outputs
    'OPTIMIZED': 0, # read transit-routing documentation, set to 0 for this project
    
    #times to test, datetime(YYYY,MM,DD,HH,MM,SS,MS)
    'first_time': datetime(2023, 3, 1, 8, 0, 0, 0), # original start time is 9am
    'end_time': datetime(2023, 3, 1, 10, 0, 0, 0), # original end time is 10am
    #'first_time': datetime(2022, 11, 24, 9, 0, 0, 0), # original start time is 9am
    #'end_time': datetime(2022, 11, 24, 10, 0, 0, 0), # original end time is 10am
    'timestep': timedelta(minutes=15), # time increments to test (original is 9am,9:20am,9:40am,10am)
    'timelimit': timedelta(hours=1), # set the max allowed total travel time in hours
    'output_fp': Path.home() / 'Documents/TransitSimData/Data' #path where you want things to output 
    }

select_tazs = ['553','1071','1005','1377'] #'288','411'

bike_settings = {
    'thresh': 5280 * 2, # set access/egress thresh
    'max_thresh': 5280 * 2 * 2, # set the max biking/walking amount
    'spd': 8,
    'mode':'bike',
    'impedance':'dist',
    'allow_wrongway':False,
    'allow_bus_to_bus':False,
    'overwrite_existing': True,
    'rail_start':False # only allow trips to start near rail stations
    }

walk_settings = {
    'thresh': 5280 * 0.625,
    'max_thresh': 5280 * 0.625 * 2, 
    'spd': 2.5,
    'mode':'walk',
    'impedance':'dist',
    'allow_wrongway':True,
    'allow_bus_to_bus':False,
    'overwrite_existing': True,
    'rail_start':False
    }

#this one assumes bike will be parked at start (does not consider availability of parking)
bikewalk_settings = {
    'thresh': (5280 * 2, 5280 * 0.625),
    'max_thresh': 5280 * (2+0.625),
    'spd': (8,2.5),
    'mode':'bikewalk',
    'impedance':'dist',
    'allow_wrongway':(False,True),
    'allow_bus_to_bus':False,
    'overwrite_existing': True,
    'rail_start':False
    }


In [ ]:
cwd = os.getcwd()
os.chdir(Path.home()/'Documents/GitHub/transit-routing')

In [ ]:
print('Running raptor algorithm')


select_tazs = ['553']
raptor_settings = settings
mode_specific = bikewalk_settings

#import routes for naming
route = pd.read_csv(raptor_settings['gtfs_fp'] / 'route.txt')

#retrieve info from dicts
mode = mode_specific['mode']
impedance = mode_specific['impedance']

#deal with travel speeds if there are two
if isinstance(mode_specific['spd'],tuple):
    spd1 = mode_specific['spd'][0]
    spd2 = mode_specific['spd'][1]
else:
    spd1 = mode_specific['spd']
    spd2 = mode_specific['spd']

print(f'First leg speed is {spd1} and last leg speed is {spd2}')

#get filepaths for select tazs
#set to where trips are stored
trips_dir = raptor_settings['output_fp'] / f'{mode}_{impedance}/trips'
all_trips = [trips_dir / f'{taz}.parquet' for taz in select_tazs]

#import network files
stops_file, trips_file, stop_times_file, transfers_file, stops_dict, stoptimes_dict, footpath_dict, routes_by_stop_dict, idx_by_route_stop_dict, routesindx_by_stop_dict = read_testcase(
    raptor_settings['NETWORK_NAME'])

#print details
print_network_details(transfers_file, trips_file, stops_file)

#get list of times
start_times = get_times(raptor_settings['first_time'],raptor_settings['end_time'],raptor_settings['timestep'])




In [ ]:
trips_fp = Path.home() / 'Documents/TransitSimData/Data/bike_dist/trips/553.parquet'
trips = pd.read_parquet(trips_fp)
#trips = trips[trips['dest_taz']=='658']

In [ ]:
trips

In [ ]:

start_time = start_times[0]
trips_fp = Path.home() / 'Documents/TransitSimData/Data/bike_dist/trips/1071.parquet'
    
#get start taz name
taz_name = trips_fp.parts[-1].split('.')[0]

#make file name
time_name = f'{start_time.hour}_{start_time.minute}'

#record starting time
time_start = time.time()

#import trips file
trips = pd.read_parquet(trips_fp)

#check mode restrictions
if mode_specific['allow_bus_to_bus'] == False:
        route_and_stop = gpd.read_file(raptor_settings['output_fp']/'base_layers.gpkg',layer='route_and_stop')[['stop_id','route_type']]
        trips = pd.merge(trips,route_and_stop,left_on='src_stop',right_on='stop_id',how='left')
        trips = pd.merge(trips,route_and_stop,left_on='dest_stop',right_on='stop_id',how='left')
        check = (trips['route_type_y'] == 3) & (trips['route_type_x'] == 3)
        trips = trips[-check]
        print(f"{check.sum()} trips were bus to bus")
        trips.drop(columns=['stop_id_x','stop_id_y','route_type_y','route_type_x'],inplace=True)



#get actual first leg time (using feet and miles per hour)
trips[f'{impedance}_to_time'] = pd.to_timedelta(trips[f'{impedance}_first_leg'] / 5280 / spd1 * 60 * 60, unit='s')#.dt.round(datetime.timedelta(minutes=1))

#find actual final leg time
trips[f'{impedance}_from_time'] = pd.to_timedelta(trips[f'{impedance}_last_leg'] / 5280 / spd2 * 60 * 60, unit='s')#.dt.round(datetime.timedelta(minutes=1))

#get arrival time (to nearest minute) at first transit stop (needs to be datetime)
trips['arrival_time'] = (start_time + trips[f'{impedance}_to_time']).dt.round(timedelta(minutes=1))

#make empty cols to store raptor outputs
trips['status'] = pd.Series(dtype=str)
trips['transit_time'] = pd.Series(dtype='timedelta64[ns]')
trips['travel_time'] = pd.Series(dtype='timedelta64[ns]')
trips['num_transfers'] = pd.Series(dtype=int)
trips['edge_list'] = pd.Series(dtype=object)

#create a new raptor df for calucating the all unique src/dest transit stops with same arrival time at first stop
raptor_df = trips[['src_stop','dest_stop','arrival_time']].drop_duplicates().reset_index(drop=True)


In [ ]:
#trips = trips[trips['dest_taz']=='658']
#trips = trips[trips['src_stop']=='1074']
#trips = trips[trips['dest_stop']=='3200']

In [ ]:
trips

In [ ]:
trips.drop_duplicates(inplace=True)


In [ ]:

#import solved raptor trips to see if it's already been calculated 
if ((raptor_settings['output_fp'] / f"raptor_dict_{raptor_settings['MAX_TRANSFER']}_transfers.pkl").exists()):
        with (raptor_settings['output_fp'] / f"raptor_dict_{raptor_settings['MAX_TRANSFER']}_transfers.pkl").open(mode='rb') as fh:
                pareto_dict = pickle.load(fh)
        #remove rows from raptor_df if already solved
        raptor_df['tup'] = list(zip(raptor_df['src_stop'].astype(int),raptor_df['dest_stop'].astype(int),raptor_df['arrival_time']))
        previously_calculated = raptor_df['tup'].isin(list(pareto_dict.keys()))
        raptor_df = raptor_df[-previously_calculated]
else:
        #initialize an empty dict to store pareto results
        pareto_dict = {}

if raptor_df.shape[0] != 0:
        print(f'Performing transit routing from {taz_name} at {start_time}:')
        #should loop through all unique times instead of every row (round arrival time to nearest minute)
        for row in tqdm(raptor_df.itertuples(),total=raptor_df.shape[0]):
                #for row in raptor_df.itertuples():
                
                #pull out the three inputs needed for raptor
                SOURCE = int(row[1])
                DESTINATION = int(row[2])
                D_TIME = row[3]

                #run standard raptor algorithm
                output, pareto_set = raptor(SOURCE, DESTINATION, D_TIME, raptor_settings['MAX_TRANSFER'], 
                                raptor_settings['WALKING_FROM_SOURCE'], raptor_settings['CHANGE_TIME_SEC'], raptor_settings['PRINT_ITINERARY'],
                                routes_by_stop_dict, stops_dict, stoptimes_dict, footpath_dict, idx_by_route_stop_dict)
                
                #store pareto set in dict for next step
                pareto_dict[(SOURCE,DESTINATION,D_TIME)] = pareto_set
else:
        print(f"RAPTOR solution already found from {taz_name} at {start_time}")


In [ ]:

#need to change this from iterrows to concat instead
print(f'Solving shortest {mode} + transit routing from {taz_name} at {start_time}:')
for row in tqdm(trips.itertuples(),total=trips.shape[0]):
#for row in trips.itertuples():
        
        #pull out the three inputs needed for raptor
        SOURCE = int(row[3])
        DESTINATION = int(row[4])
        D_TIME = row[11]
        
        #get pareto set from the pareto dict
        pareto_set = pareto_dict.get((SOURCE,DESTINATION,D_TIME),'Error')

        #skip to next if no solutions
        #come back to this
        if pareto_set == None:
                trips.at[row[0],'status'] = 'not possible'
                continue
        elif pareto_set == 'Error':
                print('Error')
                break

        #create empty list for storing pareto results
        shortest_time = []
        
        #go through each pareto optimal for the given number of transfers
        for pareto_optimal in pareto_set:
        
                #pull out the number of transfers and the edge list
                num_transfers = pareto_optimal[0]
                edges = pareto_optimal[1]
        
        #get rid of final walking leg (will be replaced with network results)
        while edges[-1][0] == 'walking':
                edges = edges[:-1]

        #get total travel time from start (time at final egress + last bike/walk leg - departure time)
        travel_time = edges[-1][3] + row[10] - start_time

        #get total transit travel time (final egress - departure time from first transit stop) 
        transit_time = edges[-1][3] - D_TIME

        #store as list
        candidate = [num_transfers,travel_time,transit_time,edges]

        #only retain shortest travel time
        if len(shortest_time) == 0:
                shortest_time = candidate
        elif candidate[1] < shortest_time[1]:
                shortest_time = candidate
        
        #update trips dataframe with results from raptor 
        trips.at[row[0],'num_transfers'] = shortest_time[0]
        trips.at[row[0],'travel_time'] = shortest_time[1]
        trips.at[row[0],'transit_time'] = shortest_time[2]
        
        #initialize edge list for storing legs of transit trip
        edge_list = []
        
        #track the wait time
        wait_time = timedelta(minutes=0)

        #store the first arrival time
        arrive = D_TIME

        #track transfer time
        transfer_time = timedelta(minutes=0)

        #edge list structure
        #leg[0] : ""walking" (str) or the boarding time (datetime) if transit
        #leg[1] : starting transit stop
        #leg[2] : ending transit stop
        #leg[3] : if walking = travel time and if transit = time at egress
        #leg[4] : only for transit = route and trip number

        #transit tup to output for edge list
        #(start stop, end stop, egress time - boarding time (travel time), route/trip, transit mode)

        #go through each leg in edge list and format
        for leg in shortest_time[3]:
        
                #format transit legs
                if leg[0] != 'walking':
                        
                        #calculate wait time for segment (time at boarding - time arrived)
                        segment_wait_time = leg[0] - arrive

                        #add to total weight time
                        wait_time += segment_wait_time
                        
                        #replace with the next arrival time at the next stop
                        arrive = leg[3]

                        #extract route_id
                        route_id = str.split(leg[4],'_')[0]
                        
                        #get transit mode from the transit routes csv
                        route_type = route[route['new_route_id'].astype(str)==route_id]['route_type'].item()

                        #get transit line name
                        name = route[route['new_route_id'].astype(str)==route_id]['route_long_name'].item()      
                        
                        #format transit tuple to add to the edge list
                        #TODO work with prateek on ensuring that the newly labeled routes and stops are labeled in all GTFS files
                        if (route_type == 1) | (route_type == '1'):
                                tup = (leg[1],leg[2],leg[3]-leg[0],leg[4],'rail',name)
                        elif (route_type == 3) | (route_type == '3'):
                                tup = (leg[1],leg[2],leg[3]-leg[0],leg[4],'bus',name) 

                #format walk legs
                else:
                        #format the walking tuple and add to edge list
                        tup = (leg[1],leg[2],leg[3],'walking')
                        
                        #track transfer time (so it can be subtracted from wait time)
                        transfer_time += leg[3]

        #add tuple to edge list
        edge_list.append(tup)
        
        #get total wait time
        trips.at[row[0],'wait_time'] = wait_time - transfer_time
        
        #get total transfer time
        trips.at[row[0],'transfer_time'] = transfer_time

        #add edge list to trips
        trips.at[row[0],'edge_list'] = edge_list
        
        #set initial success message
        trips.at[row[0],'status'] = 'success'
        
        # removed these two for now
        # #check travel time
        # if travel_time >= raptor_settings['timelimit']:
        #     trips.at[row[0],'status'] = 'time limit exceeded'
        
        # #check mode restrictions
        # if not mode_specific['allow_bus_to_bus']:
        #     if len([x for x in edge_list if x[-1] == 'bus']) > 1:
        #         trips.at[row[0],'status'] = 'two buses'

#embed start time
trips['start_time'] = start_time



In [ ]:
trips.loc[trips.travel_time.idxmin(),:]

In [ ]:
row[10]

In [ ]:
#imports
import geopandas as gpd
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import fiona
import numpy as np

#import custom functions
from helper_functions import process_results, load_files

'''
Notes:

take more of a sql approach update/merge?

have taz centroid/polygon layer instead seperating into many layers

when merging, it's duplicating the dest_taz column

'''

def viz_and_metrics(settings:dict,impedance:str,mode:str,list_tazs:list):

    # #get list of tazs
    # list_tazs = settings['output_fp'].glob(f'{mode}_{impedance}/raptor_results/*')
    # list_tazs = [x.parts[-1] for x in list_tazs]

    # save to same .gpkg as taz name
    # #make the vis folder
    # if not (settings['output_fp'] / f'{mode}_{impedance}/visuals').exists():
    #     (settings['output_fp'] / f'{mode}_{impedance}/visuals').mkdir()

    for taz in tqdm(list_tazs):
        
        #import taz polygons
        tazs = gpd.read_file(settings['output_fp'] / 'base_layers.gpkg',layer='tazs')[[settings['keyid'],'geometry']]

        #import taz centroids
        centroids = gpd.read_file(settings['output_fp'] / 'base_layers.gpkg',layer='centroids')[[settings['keyid'],'geometry']] 

        #map all transit routes used
        transit_shed(settings,impedance,mode,taz)

        #import walkable/bikable tazs
        #bike_tazs = gpd.read_file(settings['output_fp'] / f'{mode}_{impedance}/{taz}.gpkg',layer=f'{mode}able_tazs_polygons',ignore_geometry=True)

        #get filepaths for all departure times
        dep_times = settings['output_fp'].glob(f'{mode}_{impedance}/raptor_results/{taz}/*.pkl')

        #initialize empty dataframe for storing all the trip data per departure time
        all_trips = pd.DataFrame()

        #loop through each departure time
        for dep_time in dep_times:

            #get departure time text
            #dep = dep_time.parts[-1].split('.pkl')[0]

            #read in results file and find the least time route per departure time
            with fp.open(mode='rb') as fh:
                trips = pickle.load(fh)
            #only get successful trips
            trips = trips[trips['status']=='success']
            #groupby to get minimum
            trips = trips.loc[trips.groupby(['src_taz','dest_taz'])['travel_time'].idxmin().to_list(),:]

            #convert time cols from datetime to total minutes (round to 1 decimal)
            time_cols = trip_df.columns[trip_df.dtypes == 'timedelta64[ns]']
            for time_col in time_cols:
                trip_df[time_col] = trip_df[time_col].apply(lambda x: round(x.total_seconds() / 60,1))

            #append to all_trips dataframe
            all_trips = pd.concat([all_trips,trip_df],ignore_index=True)

        #get average transit time, travel time, and wait time
        avg_transit_time = all_trips.groupby('dest_taz')['transit_time'].mean()
        avg_travel_time = all_trips.groupby('dest_taz')['travel_time'].mean()
        avg_wait_time = all_trips.groupby('dest_taz')['wait_time'].mean()

        #get minimum number of transfers
        min_transfers = all_trips.groupby('dest_taz')['num_transfers'].min()

        # go through the edge list and concatanate all the transit modes together  
        for idx, row in all_trips.iterrows():
            
            #list comp to get list of modes for each trip
            modes = [edge[-1] for edge in row['edge_list'] if (edge[-1] == 'bus') or (edge[-1] == 'rail')]
            
            #turn to series
            modes = pd.Series(modes)

            try:
            
                #types
                if modes.nunique() > 1:
                    types = 'Two Modes'
                elif modes[0] == 'rail':
                    types = 'Rail'
                elif modes[0] == 'bus':
                    types = 'Bus'
            except:
                print(modes)
                break
            
            #add to trip_df
            all_trips.at[idx,'types'] = types

        #get mode of transit type (bus, rail, mixed)
        print(all_trips)
        mode_type = all_trips.groupby('dest_taz')['types'].agg(pd.Series.mode)

        #if more than one mode, say mixed
        mode_type = mode_type.apply(lambda x: replace_list_with_string(x))

        #join data to tazs (i think this is where it messes up, trying map instead)
        tazs['avg_transit_time'] = tazs[settings['keyid']].map(avg_transit_time)
        tazs['avg_travel_time'] = tazs[settings['keyid']].map(avg_travel_time)
        tazs['avg_wait_time'] = tazs[settings['keyid']].map(avg_wait_time)
        tazs['min_transfers'] = tazs[settings['keyid']].map(min_transfers)
        tazs['mode_type'] = tazs[settings['keyid']].map(mode_type)
        #prolly add min/max for the times and count how many time periods taz was inaccessible
        
        #drop null rows
        tazs = tazs[-tazs.isna().any(axis=1)]

        #export
        tazs.to_file(settings['output_fp'] / f'{mode}_{impedance}/{taz}.gpkg',layer='tazs_viz')
        
        #drop the geometry column
        tazs.drop(columns=['geometry'],inplace=True)

        #join data to centroids by merging with tazs
        centroids = pd.merge(centroids,tazs,on=settings['keyid'])
        
        #drop null rows (shouldn't be any)
        centroids = centroids[-centroids.isna().any(axis=1)]

        #export
        try: 
            centroids.to_file(settings['output_fp'] / f'{mode}_{impedance}/{taz}.gpkg',layer='centroids_viz')
        except:
            print(centroids)


def replace_list_with_string(lst):
    if isinstance(lst, np.ndarray):
        return 'Mixed'
    elif isinstance(lst,list):
        return 'Mixed'
    else:
        return lst

def transit_shed(settings:dict,impedance:str,mode:str,list_taz:str):
    '''
    Creates polygon showing all the transit lines utilized using the RAPTOR outputs

    would like to also figure out a betweenness centrality like metric but that will wait
    '''

    #get filepaths
    fps = settings['output_fp'].glob(f'{mode}_{impedance}/mapped/{list_taz}/*.gpkg')

    big_df = gpd.GeoDataFrame()
    
    for fp in fps:
        #load each time
        for start_time in fiona.listlayers(fp):
        
            trip = gpd.read_file(fp,layer=start_time)
                
            #only keep transit
            trip = trip.loc[trip['mode'].isin(['rail','bus']),['mode','start_stop','end_stop','geometry']]
            
            big_df = pd.concat([big_df,trip],ignore_index=True)
        
        #drop duplicates
        big_df.drop_duplicates(['mode','start_stop','end_stop'],inplace=True)
    
    #set activee geo column
    big_df.set_geometry('geometry',inplace=True)

    #buffer because polygons faster to dissolve than linestrings
    big_df.geometry = big_df.buffer(400)
    
    #dissolve
    big_df = big_df.dissolve('mode')
    
    #export
    big_df.to_file(settings['output_fp'] / f'{mode}_{impedance}/{list_taz}.gpkg',layer='transitshed')

In [ ]:
# create transfers.txt
create_transfers(settings)

# function that creates the study area and various base layers
process_studyarea(settings)

candidate_stops_by_taz, centroids = candidate_stops(settings)

# create bike trip files
raptor_preprocessing(settings,bike_settings,select_tazs)

# create walk trip files
raptor_preprocessing(settings,walk_settings,select_tazs)

# create bikewalk trip files
raptor_preprocessing(settings,bikewalk_settings,select_tazs)
# can add more or comment out as needed

In [ ]:
#410 minutes
#change cwd Fix this later
cwd = os.getcwd()
os.chdir(Path.home()/'Documents/GitHub/transit-routing')

run_raptor(select_tazs,settings,bike_settings)
#160 minutes??
run_raptor(select_tazs,settings,walk_settings)
#430minutes
run_raptor(select_tazs,settings,bikewalk_settings)

#change back
os.chdir(cwd)

# Compact View (LTS)

In [ ]:
settings = {
    #changed network from marta to martalatest
    
    #these are for the pre-processing steps
    'gtfs_fp': Path.home() / 'Documents/GitHub/transit-routing/GTFS/martalatest', #filepath for processed GTFS files
    'gtfs_zip': Path.home() / 'Documents/GitHub/transit-routing/martalatest_gtfs.zip', #filepath for original zipped GTFS files
    'network_fp': Path.home() / 'Documents/TransitSimData/networks/final_network.gpkg', #fp to geopackage with a links and nodes layer
    'links_layer': 'lowstress_links', # name of links layer
    'nodes_layer': 'lowstress_nodes', # name of nodes layer
    'impedance': 'dist', # specify which column of the links data should be used for shortest path routing
    'service_date': date(2023,3,1),#date(2022, 11, 24), #select day to get service, a weekday was used for this study
    'crs': 'epsg:2240', # the desired projected CRS to use
    'thresh': 2.5 * 5280, #set candidate stop threshold distance in units of CRS (using bike service area from tcqsm page 5-20)
    'tazs_fp': Path.home() / 'Documents/NewBikewaySimData/Data/ARC/Model_Traffic_Analysis_Zones_2020.geojson', # filepath of TAZs or origins/POIs
    'keyid': 'OBJECTID', #column with the unique taz/origin id in it
    
    #this for making the transfers.txt file need for raptor
    'transfer_time': 2, # allowed transfer time in minutes (DOES NOT INLCUDE WAIT TIME)
    'walk_spd': 3, # assumed walking speed for transfers in miles per hour
    
    #these are specific to RAPTOR
    'NETWORK_NAME': 'martalatest',#'marta',
    'MAX_TRANSFER': 2, # no more than 1 transfer
    'WALKING_FROM_SOURCE': 0, # if true (1), person can walk to a different transit station from start station
    'CHANGE_TIME_SEC': 30, # time to enter/exit vehicle (NOT WAIT TIME)
    'PRINT_ITINERARY': 0, # if running individual raptor trips, this prints the outputs
    'OPTIMIZED': 0, # read transit-routing documentation, set to 0 for this project
    
    #times to test, datetime(YYYY,MM,DD,HH,MM,SS,MS)
    'first_time': datetime(2023, 3, 1, 8, 0, 0, 0), # original start time is 9am
    'end_time': datetime(2023, 3, 1, 10, 0, 0, 0), # original end time is 10am
    'timestep': timedelta(minutes=15), # time increments to test (original is 9am,9:20am,9:40am,10am)
    'timelimit': timedelta(hours=1), # set the max allowed total travel time in hours
    'output_fp': Path.home() / 'Documents/TransitSimData/LTS' #path where you want things to output 
    }

#select_tazs = ['1071']#['288','553','411','1071']

bikelts_settings = {
    'thresh': 5280 * 2.5, # set access/egress thresh
    'max_thresh': 5280 * 2.5 * 2, # set the max biking/walking amount
    'spd': 8,
    'mode':'bikelts',
    'impedance':'dist',
    'allow_wrongway':False,
    'allow_bus_to_bus':False,
    'overwrite_existing': True
    }

#create transfers.txt
create_transfers(settings)

# function that creates the study area and various base layers
process_studyarea(settings)

candidate_stops_by_taz, centroids = candidate_stops(settings)

raptor_preprocessing(settings,bikelts_settings,select_tazs)

#change cwd Fix this later
cwd = os.getcwd()
os.chdir(Path.home()/'Documents/GitHub/transit-routing')

run_raptor(select_tazs,settings,bikelts_settings)

#change back
os.chdir(cwd)

impedance = 'dist'
modes = ['bikelts']

for mode in modes:
    map_routes(settings,impedance,mode,select_tazs)
    viz_and_metrics(settings,impedance,mode,select_tazs)

# Using model OD data

In [ ]:
settings = {
    #changed network from marta to martalatest
    
    #these are for the pre-processing steps
    'gtfs_fp': Path.home() / 'Documents/GitHub/transit-routing/GTFS/martalatest', #filepath for processed GTFS files
    'gtfs_zip': Path.home() / 'Documents/GitHub/transit-routing/martalatest_gtfs.zip', #filepath for original zipped GTFS files
    'network_fp': Path.home() / 'Documents/TransitSimData/networks/final_network.gpkg', #fp to geopackage with a links and nodes layer
    'links_layer': 'links', # name of links layer
    'nodes_layer': 'nodes', # name of nodes layer
    'impedance': 'dist', # specify which column of the links data should be used for shortest path routing
    'service_date': date(2023,3,1),#date(2022, 11, 24), #select day to get service, a weekday was used for this study
    'crs': 'epsg:2240', # the desired projected CRS to use
    'thresh': 2.5 * 5280, #set candidate stop threshold distance in units of CRS (using bike service area from tcqsm page 5-20)
    'tazs_fp': Path.home() / 'Documents/NewBikewaySimData/Data/ARC/Model_Traffic_Analysis_Zones_2020.geojson', # filepath of TAZs or origins/POIs
    'keyid': 'OBJECTID', #column with the unique taz/origin id in it
    
    #this for making the transfers.txt file need for raptor
    'transfer_time': 2, # allowed transfer time in minutes (DOES NOT INLCUDE WAIT TIME)
    'walk_spd': 3, # assumed walking speed for transfers in miles per hour
    
    #these are specific to RAPTOR
    'NETWORK_NAME': 'martalatest',#'marta',
    'MAX_TRANSFER': 2, # no more than 1 transfer
    'WALKING_FROM_SOURCE': 0, # if true (1), person can walk to a different transit station from start station
    'CHANGE_TIME_SEC': 30, # time to enter/exit vehicle (NOT WAIT TIME)
    'PRINT_ITINERARY': 0, # if running individual raptor trips, this prints the outputs
    'OPTIMIZED': 0, # read transit-routing documentation, set to 0 for this project
    
    #times to test, datetime(YYYY,MM,DD,HH,MM,SS,MS)
    'first_time': datetime(2023, 3, 1, 4, 30, 0, 0), # original start time is 9am
    'end_time': datetime(2023, 3, 2, 00, 30, 0, 0), # original end time is 10am
    'timestep': timedelta(minutes=15), # time increments to test (original is 9am,9:20am,9:40am,10am)
    'timelimit': timedelta(hours=1), # set the max allowed total travel time in hours
    'output_fp': Path.home() / 'Documents/TransitSimData/ABM', #path where you want things to output 

    #restrict to only starting at rail station
    'rail_start' : True
    }

#bring in od data
ods = pd.read_csv(settings['output_fp']/'ods.csv')
ods['origin'] = ods['origin'].astype(str)
ods['destination'] = ods['destination'].astype(str)
#all tazs
select_tazs = ods['origin'].unique().tolist()#['1071']#['288','553','411','1071']

#check if trip is within 1 hr of the departure time (ABM specific)
ods['year'] = settings['service_date'].year
ods['month'] = settings['service_date'].month
ods['day'] = settings['service_date'].day
ods['adj_time'] = pd.to_datetime(ods[['year','month','day','hour','minute']])
ods['adjusted'] = ods['adj_time'] + pd.to_timedelta(ods['depart_time'])

# bike_settings = {
#     'thresh': 5280 * 2.5, # set access/egress thresh
#     'max_thresh': 5280 * 2.5 * 2, # set the max biking/walking amount
#     'spd': 8,
#     'mode':'bike',
#     'impedance':'dist',
#     'allow_wrongway':False,
#     'allow_bus_to_bus':False,
#     'overwrite_existing': False
#     }

walk_settings = {
    'thresh': 5280 * 0.5,
    'max_thresh': 5280 * 0.5 * 2, #set to twice
    'spd': 3,
    'mode':'walk',
    'impedance':'dist',
    'allow_wrongway':True,
    'allow_bus_to_bus':True,
    'overwrite_existing': True
    }

#this one assumes bike will be parked at start (removes bus stops from first mile that aren't next to rail)
bikewalk_settings = {
    'thresh': (5280 * 2.5, 5280 * 0.5),
    'max_thresh': 5280 * (3),
    'spd': (8,2.5),
    'mode':'bikewalk',
    'impedance':'dist',
    'allow_wrongway':(False,True),
    'allow_bus_to_bus':True,
    'overwrite_existing': True
    }


In [ ]:

#create transfers.txt
create_transfers(settings)

# function that creates the study area and various base layers
process_studyarea(settings)

candidate_stops_by_taz, centroids = candidate_stops(settings)

raptor_preprocessing(settings,bikewalk_settings,select_tazs,ods)
raptor_preprocessing(settings,walk_settings,select_tazs,ods)


In [ ]:

#change cwd Fix this later
cwd = os.getcwd()
os.chdir(Path.home()/'Documents/GitHub/transit-routing')

#run_raptor(select_tazs,settings,bikewalk_settings,ods)
run_raptor(select_tazs,settings,walk_settings,ods)

#change back
os.chdir(cwd)


In [ ]:

impedance = 'dist'
modes = ['walk','bikewalk']

for mode in modes:#
    map_routes(settings,impedance,mode,select_tazs)
    viz_and_metrics(settings,impedance,mode,select_tazs)
